In [2]:
from itertools import product
import re

# ---- Parser / evaluator utilities ----

def tokenize(formula):
    """Turn formula string into tokens."""
    # normalize spaces
    s = formula.replace(" ", "")
    # token patterns: symbols (letters+digits+_), parentheses, operators ->, <->, ~, &, |
    token_spec = r"(<->|->|~|\(|\)|&|\||[A-Za-z_][A-Za-z0-9_]*)"
    tokens = re.findall(token_spec, s)
    if "".join(tokens) != s:
        raise ValueError(f"Failed to tokenize formula: {formula!r}")
    return tokens

def infix_to_postfix(tokens):
    """
    Shunting-yard algorithm supporting unary ~, binary &, |, ->, <->.
    Precedence (high to low): ~ , & , | , -> , <->
    Associativity: ~ (right/unary), & left, | left, -> right, <-> left
    """
    prec = {'~': 5, '&': 4, '|': 3, '->': 2, '<->': 1}
    assoc = {'~': 'right', '&': 'left', '|': 'left', '->': 'right', '<->': 'left'}
    output = []
    stack = []
    i = 0
    while i < len(tokens):
        t = tokens[i]
        if re.fullmatch(r"[A-Za-z_][A-Za-z0-9_]*", t):  # symbol
            output.append(t)
        elif t == '(':
            stack.append(t)
        elif t == ')':
            while stack and stack[-1] != '(':
                output.append(stack.pop())
            if not stack:
                raise ValueError("Mismatched parentheses")
            stack.pop()  # pop '('
        else:  # operator
            # handle unary ~: when token is ~ it's unary; that's fine
            while stack and stack[-1] != '(':
                top = stack[-1]
                if ((assoc[t] == 'left' and prec[t] <= prec[top]) or
                    (assoc[t] == 'right' and prec[t] < prec[top])):
                    output.append(stack.pop())
                else:
                    break
            stack.append(t)
        i += 1
    while stack:
        if stack[-1] in ('(', ')'):
            raise ValueError("Mismatched parentheses")
        output.append(stack.pop())
    return output

def eval_postfix(postfix, model):
    """Evaluate postfix expression given model: dict symbol->bool"""
    def implies(p, q):
        return (not p) or q
    def bicond(p, q):
        return p == q

    stack = []
    for tok in postfix:
        if re.fullmatch(r"[A-Za-z_][A-Za-z0-9_]*", tok):
            if tok not in model:
                raise KeyError(f"Symbol {tok} not in model")
            stack.append(bool(model[tok]))
        elif tok == '~':
            if not stack:
                raise ValueError("Bad syntax: unary operator with empty stack")
            v = stack.pop()
            stack.append(not v)
        else:
            # binary op
            if len(stack) < 2:
                raise ValueError("Bad syntax: binary operator with insufficient operands")
            b = stack.pop()
            a = stack.pop()
            if tok == '&':
                stack.append(a and b)
            elif tok == '|':
                stack.append(a or b)
            elif tok == '->':
                stack.append(implies(a, b))
            elif tok == '<->':
                stack.append(bicond(a, b))
            else:
                raise ValueError(f"Unknown operator {tok}")
    if len(stack) != 1:
        raise ValueError("Bad syntax: leftover stack after evaluation")
    return stack[0]

def parse_formula(formula):
    """Tokenize and convert to postfix once (for efficiency)."""
    tokens = tokenize(formula)
    postfix = infix_to_postfix(tokens)
    return postfix

# ---- Main entailment checker ----

def check_entailment(symbols, kb_formulas, query_formula, verbose=True):
    """
    Check whether KB entails query by enumerating all models (truth table).
    symbols: list of symbol names (strings)
    kb_formulas: list of formula strings (KB sentences)
    query_formula: formula string
    Returns: True if KB |= query, False otherwise.
    """
    # parse formulas to postfix once
    kb_postfix = [parse_formula(f) for f in kb_formulas]
    query_postfix = parse_formula(query_formula)

    rows = []
    if verbose:
        header = f"{' | '.join(symbols)}  |  " + "  |  ".join([f"{f}" for f in kb_formulas]) + "  |  " + query_formula
        print(header)
        print("-" * len(header))

    # iterate models
    for vals in product([False, True], repeat=len(symbols)):
        model = dict(zip(symbols, vals))
        # evaluate KB sentences
        kb_values = [eval_postfix(pf, model) for pf in kb_postfix]
        kb_true = all(kb_values)
        query_value = eval_postfix(query_postfix, model)
        rows.append((model, kb_values, kb_true, query_value))
        if verbose:
            sym_vals = " | ".join('T' if model[s] else 'F' for s in symbols)
            kb_vals_str = " | ".join('T' if v else 'F' for v in kb_values)
            qstr = 'T' if query_value else 'F'
            print(f"{sym_vals}  |  {kb_vals_str}  |  {qstr}")

    # collect models where KB is true
    kb_models = [r for r in rows if r[2]]
    if verbose:
        print("\nModels where KB is true:")
        if not kb_models:
            print("  (none) -- KB is unsatisfiable")
        else:
            for mod, kbvals, _, qv in kb_models:
                mv = ", ".join(f"{k}={'T' if v else 'F'}" for k, v in mod.items())
                print(f"  {mv}  (Query={ 'T' if qv else 'F' })")

    # entailment: query true in all KB models
    if not kb_models:
        # By classical logic, unsatisfiable KB entails every formula (vacuous truth).
        entailed = True
    else:
        entailed = all(r[3] for r in kb_models)

    if verbose:
        print("\nEntailment result:")
        if entailed:
            print("  KB ⊨ query  — query is entailed by KB.")
        else:
            print("  KB ⊭ query  — query is NOT entailed by KB (countermodel exists).")

    return entailed

# ---- Example: use the burglary example from your prompt ----
if __name__ == "__main__":
    symbols = ['B', 'A', 'C']
    KB = ["B -> A", "A -> C", "B"]
    query = "C"
    result = check_entailment(symbols, KB, query, verbose=True)
    print("\nFinal answer:", result)


B | A | C  |  B -> A  |  A -> C  |  B  |  C
-------------------------------------------
F | F | F  |  T | T | F  |  F
F | F | T  |  T | T | F  |  T
F | T | F  |  T | F | F  |  F
F | T | T  |  T | T | F  |  T
T | F | F  |  F | T | T  |  F
T | F | T  |  F | T | T  |  T
T | T | F  |  T | F | T  |  F
T | T | T  |  T | T | T  |  T

Models where KB is true:
  B=T, A=T, C=T  (Query=T)

Entailment result:
  KB ⊨ query  — query is entailed by KB.

Final answer: True
